In [20]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, ReLU, Flatten, Dense, Softmax
from tensorflow.keras.optimizers import Adam
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import BatchNormalization, Dropout

In [2]:
tf.__version__

'2.14.0'

# Data Pre-Processing

Open **dan_train.csv** file and split the games into a list.
Every row of csv: `DL0000000001,B,B[pd],W[dp],B[pp],W[dc],B[de],...`.

Columns are:

    1. DL0000000001: Game ID
    2. B: Player's color
    3-... : Moves
    
We cropped only the moves to game list as:

In [5]:
df = open('/content/drive/MyDrive/AI/AIcupTutorial-main/CSVs/Tutorial_dan_train.csv').read().splitlines()
games = [i.split(',',2)[-1] for i in df]

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Create a dictionary to convert the coordinates from characters to numbers

In [9]:
chars = 'abcdefghijklmnopqrs'
coordinates = {k:v for v,k in enumerate(chars)}
chartonumbers = {k:v for k,v in enumerate(chars)}
coordinates

{'a': 0,
 'b': 1,
 'c': 2,
 'd': 3,
 'e': 4,
 'f': 5,
 'g': 6,
 'h': 7,
 'i': 8,
 'j': 9,
 'k': 10,
 'l': 11,
 'm': 12,
 'n': 13,
 'o': 14,
 'p': 15,
 'q': 16,
 'r': 17,
 's': 18}

We decided to build a DCNN model in this tutorial. We create data samples by using every move in every game, meaning that the target is to predict the next move by feeding the previous state of the table in every game for every move. Therefore, we can collect much more data samples from games.

For the simplicity, we used 4 dimensional feature map to represent the data as below:
 1. Positions of black stones: mark them as 1 and the rest of the table as 0
 2. Positions of white stones: mark them as 1 and the rest of the table as 0
 3. Empty areas of the table: mark the empty areas as 1 and occupied areas as 0
 4. The last move in the table: mark the position of the last move as 1 and the rest as 0

Target value is a number between 0-361(19\*19). Later this will be one-hot encoded.

In [6]:
def prepare_input(moves):
    x = np.zeros((19,19,4))
    for move in moves:
        color = move[0]
        column = coordinates[move[2]]
        row = coordinates[move[3]]
        if color == 'B':
            x[row,column,0] = 1
            x[row,column,2] = 1
        if color == 'W':
            x[row,column,1] = 1
            x[row,column,2] = 1
    if moves:
        last_move_column = coordinates[moves[-1][2]]
        last_move_row = coordinates[moves[-1][3]]
        x[row,column,3] = 1
    x[:,:,2] = np.where(x[:,:,2] == 0, 1, 0)
    return x

def prepare_label(move):
    column = coordinates[move[2]]
    row = coordinates[move[3]]
    return column*19+row

In [7]:
# Check how many samples can be obtained
n_games = 0
n_moves = 0
for game in games:
    n_games += 1
    moves_list = game.split(',')
    for move in moves_list:
        n_moves += 1
print(f"Total Games: {n_games}, Total Moves: {n_moves}")

Total Games: 1280, Total Moves: 297110


The code below is run for baseline model only by using only the first 500 games from the dataset. You might need to create a data generator to use complete dataset. Otherwise your RAM might not enough to store all (If you run the code on free version of Google Colab, it will crash above 500 game samples).

In [10]:
x = []
y = []
for game in games[:500]:
    moves_list = game.split(',')
    for count, move in enumerate(moves_list):
        x.append(prepare_input(moves_list[:count]))
        y.append(prepare_label(moves_list[count]))
x = np.array(x)
y = np.array(y)

In [11]:
x.shape

(118278, 19, 19, 4)

In [12]:
y.shape

(118278,)

In [13]:
y_one_hot = tf.one_hot(y, depth=19*19)

Dataset splitting: 90% Training, 10% validation

In [14]:
x_train, x_val, y_train, y_val = train_test_split(x, y_one_hot.numpy(), test_size=0.10)

# Training

### Simple DCNN Model:

In [15]:
def create_model():
    inputs = Input(shape=(19, 19, 4))
    outputs = Conv2D(kernel_size=7, filters=32, padding='same', activation='relu')(inputs)
    outputs = Conv2D(kernel_size=7, filters=32, padding='same', activation='relu')(outputs)
    outputs = Conv2D(kernel_size=5, filters=32, padding='same', activation='relu')(outputs)
    outputs = Conv2D(kernel_size=5, filters=32, padding='same', activation='relu')(outputs)
    outputs = Conv2D(kernel_size=3, filters=32, padding='same', activation='relu')(outputs)
    outputs = Conv2D(kernel_size=3, filters=1, padding='same', activation='relu')(outputs)
    outputs = Flatten()(outputs)
    outputs = Softmax()(outputs)
    model = Model(inputs, outputs)

    opt = Adam(learning_rate=0.001)
    model.compile(optimizer=opt,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

In [16]:
model = create_model()
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 19, 19, 4)]       0         
                                                                 
 conv2d (Conv2D)             (None, 19, 19, 32)        6304      
                                                                 
 conv2d_1 (Conv2D)           (None, 19, 19, 32)        50208     
                                                                 
 conv2d_2 (Conv2D)           (None, 19, 19, 32)        25632     
                                                                 
 conv2d_3 (Conv2D)           (None, 19, 19, 32)        25632     
                                                                 
 conv2d_4 (Conv2D)           (None, 19, 19, 32)        9248      
                                                                 
 conv2d_5 (Conv2D)           (None, 19, 19, 1)         289   

In [17]:
history = model.fit(
    x = x_train,
    y = y_train,
    batch_size = 128,
    epochs = 10,
    validation_data=(x_val, y_val),
)

Epoch 1/10
832/832 [==============================] - 1019s 1s/step - loss: 4.8618 - accuracy: 0.0479 - val_loss: 4.4281 - val_accuracy: 0.0665
Epoch 2/10
832/832 [==============================] - 1010s 1s/step - loss: 4.3804 - accuracy: 0.0835 - val_loss: 4.3016 - val_accuracy: 0.0833
Epoch 3/10
832/832 [==============================] - 997s 1s/step - loss: 4.2863 - accuracy: 0.0978 - val_loss: 4.2518 - val_accuracy: 0.0962
Epoch 4/10
832/832 [==============================] - 995s 1s/step - loss: 4.2142 - accuracy: 0.1092 - val_loss: 4.1915 - val_accuracy: 0.0990
Epoch 5/10
832/832 [==============================] - 995s 1s/step - loss: 4.1390 - accuracy: 0.1239 - val_loss: 4.1078 - val_accuracy: 0.1202
Epoch 6/10
832/832 [==============================] - 1034s 1s/step - loss: 4.0580 - accuracy: 0.1381 - val_loss: 4.0671 - val_accuracy: 0.1312
Epoch 7/10
832/832 [==============================] - 1007s 1s/step - loss: 3.9869 - accuracy: 0.1523 - val_loss: 4.0033 - val_accuracy: 0.

In [18]:
model.save('/content/drive/MyDrive/AI/AIcupTutorial-main/model_dan_tutorial.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


## ALL DONE!

For using the model and creating a submission file, follow the notebook **Create Public Upload CSV.ipynb**

# End of Tutorial

You are free to use more modern NN architectures, a better pre-processing, feature extraction methods to achieve much better accuracy!

In [ ]:
def create_improved_model():
    inputs = Input(shape=(19, 19, 4))

    # Added more convolutional layers with increased filter numbers
    conv1 = Conv2D(kernel_size=7, filters=64, padding='same', activation='relu')(inputs)
    conv1 = BatchNormalization()(conv1)
    conv1 = Dropout(0.3)(conv1)

    conv2 = Conv2D(kernel_size=5, filters=128, padding='same', activation='relu')(conv1)
    conv2 = BatchNormalization()(conv2)
    conv2 = Dropout(0.4)(conv2)

    conv3 = Conv2D(kernel_size=3, filters=256, padding='same', activation='relu')(conv2)
    conv3 = BatchNormalization()(conv3)
    conv3 = Dropout(0.5)(conv3)

    conv4 = Conv2D(kernel_size=3, filters=512, padding='same', activation='relu')(conv3)
    conv4 = BatchNormalization()(conv4)
    conv4 = Dropout(0.5)(conv4)

    outputs = Conv2D(kernel_size=1, filters=1, padding='same', activation='linear')(conv4)
    outputs = Flatten()(outputs)
    outputs = Dense(361, activation='softmax')(outputs)  # Changed the activation function

    model = Model(inputs, outputs)

    opt = Adam(learning_rate=0.001)
    model.compile(optimizer=opt,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

improved_model = create_improved_model()
improved_model.summary()

# Training the improved model
improved_history = improved_model.fit(
    x=x_train,
    y=y_train,
    batch_size=128,
    epochs=15,  # Increased the number of epochs for more training
    validation_data=(x_val, y_val)
)

# Save the improved model
improved_model.save('/content/drive/MyDrive/AI/AIcupTutorial-main/improved_model_dan_tutorial.h5')

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 19, 19, 4)]       0         
                                                                 
 conv2d_7 (Conv2D)           (None, 19, 19, 64)        12608     
                                                                 
 batch_normalization (Batch  (None, 19, 19, 64)        256       
 Normalization)                                                  
                                                                 
 dropout (Dropout)           (None, 19, 19, 64)        0         
                                                                 
 conv2d_8 (Conv2D)           (None, 19, 19, 128)       204928    
                                                                 
 batch_normalization_1 (Bat  (None, 19, 19, 128)       512       
 chNormalization)                                          